## Imports

In [1]:
# Import the usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import Olist data
from olist.data import Olist
from olist.order import Order

# Olist's Net Promoter Score (NPS) 🔥

The **Net Promoter Score (NPS)** of a service answers the following question:

> How likely is it that you would recommend our company/product/service to a friend or colleague?

For a service rated between 1 and 5 stars, like Olist, we can **classify customers into three categories** based on their answers:
- ✅ **Promoters**: customers who answered  with a score of 5
- 😴 **Passive**: customers who answered with a score of 4 
- 😡 **Detractors**: customers who answered with a score between 1 and 3 (inclusive)

<br>

👉 NPS is computed by subtracting the percentage of customers who are **detractors** from the percentage of customers who are **promoters**.

> NPS  
= % Promoters - % Detractors   
= (# Promoter - # Detractors) / # Reviews  
= (# 5 stars - # <4 stars) / # Reviews

## Computing the Overall NPS Score of Olist

In [2]:
data = Olist().get_data()
orders = Order().get_training_data()

❓Create a function that converts `review_score` into `nps_class`. `nps_class` should be a **classification** depending on the `review_score`, so there are 3 possibilities:

- `review_score` is **5** 👉 `nps_class` is **1** (promoter)
- `review_score` is **4** 👉 `nps_class` is **0** (passive)
- `review_score` is **3** or less 👉 `nps_class` is **-1** (detractor)

In [4]:
def promoter_score(review_score):
    if review_score == 5:
        return 1
    elif review_score == 4:
        return 0
    return -1
orders['nps_class'] = orders['review_score'].apply(promoter_score)

💡Let's try to rewrite this function into a single line of code that achieves the same result 😏

There are **several** ways to do it! Let's look at some of them, then we can compare their execution times to that of our function to see which one is more efficient ⏱️

Two general principles when it comes to programming/coding are:
- `KISS`: **K**eep **I**t **S**imple and **S**mart
- `DRY`: **D**on't **R**epeat **Y**ourself 😉

<details>
    <summary>💡Hint</summary>

Use the following methods and use `%time` to compare their execution times:
- `.apply()` with the function you wrote above
- `.map()` or `.apply()` with a `lambda` function
- `.loc[]` with boolean indexing
- `np.select()` with matching conditions

</details>    

In [5]:
data = {
    'review_score': [5, 4, 3, 2, 1, 5, 4, 3, 5, 5, 1]
}
orders = pd.DataFrame(data)
orders['nps_class'] = np.select(
    [orders['review_score'] == 5, orders['review_score'] == 4],
    [1, 0],
    default=-1
)
print(orders)

%time orders['nps_class'] = np.select([orders['review_score'] == 5, orders['review_score'] == 4], [1, 0], default=-1)

    review_score  nps_class
0              5          1
1              4          0
2              3         -1
3              2         -1
4              1         -1
5              5          1
6              4          0
7              3         -1
8              5          1
9              5          1
10             1         -1
CPU times: user 943 µs, sys: 119 µs, total: 1.06 ms
Wall time: 5.38 ms


In [6]:
%%time
# Even more concisely with np.select()
orders['promoter_class'] = np.select([orders.review_score >= 4], [orders.review_score - 4], -1)

CPU times: user 1.37 ms, sys: 404 µs, total: 1.77 ms
Wall time: 2.01 ms


**A Note About `.apply()`**

Consider the following examples:

```python
df.apply(lambda col: col.max(), axis = 0)
df.apply(lambda row: row['A'] + row['B'], axis = 1)
```

These operations look similar because they both use `.apply()`, but one is much slower than the other. The data layout for Pandas DataFrames is **column-major** (read more [here](https://en.wikipedia.org/wiki/Row-_and_column-major_order)), which means that column-wise operations are always going to be faster than row-wise operations. The second example above uses `axis=1`, making it a row-wise operation, which would be more appropriate for **row-major** data layouts such as NumPy arrays.

For small amounts of data, this difference is irrelevant, but when you start working with huge datasets this will probably make a big difference. For big datasets, you're likely to notice that using `.loc[]`, `np.select()` or `np.apply_along_axis()` will run faster on Pandas DataFrames when applying a function on every row.

It's always good to understand how your data is stored before you access it!

👇 Now that you have the different promoter scores, you can compute `Olist's NPS`.

In [7]:
n_promoters = (orders['nps_class'] == 1).sum()
n_passives = (orders['nps_class'] == 0).sum()
n_detractors = (orders['nps_class'] == -1).sum()

nps = ((n_promoters - n_detractors) / len(orders)) * 100
print(f"Net Promoter Score (NPS): {nps:.2f}")

Net Promoter Score (NPS): -9.09


## NPS per Customer State

👇 Here is the part of Olist's DB schema that is relevant for this section, to help you have an overview of things.

<img src="https://wagon-public-datasets.s3-eu-west-1.amazonaws.com/04-Decision-Science/02-Statistical-Inference/olist_schema.png" width=750>

### What is the average review score per state?

❓First, create the dataset required for computation

In [13]:
orders_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_orders_dataset.csv')
customers_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_customers_dataset.csv')
reviews_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_order_reviews_dataset.csv')

merged_df = pd.merge(orders_df, customers_df, how='left', left_on='customer_id', right_on='customer_id')
merged_df = pd.merge(merged_df, reviews_df, how='left', left_on='order_id', right_on='order_id')

avg_review_score_per_state = merged_df.groupby('customer_state')['review_score'].mean().reset_index()

👉 Now, we can aggregate this dataset per  `customer_state` using any aggregation method of our choice :)

❓ Let's start with the average review score: compute the average `review_score` per `customer_state`.

*Hints:* try to tackle this question using three different methods:
- with `.mean()`
- then with `.apply()`
- and eventually the `.agg()`

In [15]:
avg_review_score_per_state = merged_df.groupby('customer_state')['review_score'].mean().reset_index()
avg_review_score_per_state.rename(columns={'review_score': 'average_review_score'}, inplace=True)

In [16]:
avg_review_score_per_state_apply = merged_df.groupby('customer_state')['review_score'].apply(lambda x: x.mean()).reset_index()
avg_review_score_per_state_apply.rename(columns={'review_score': 'average_review_score'}, inplace=True)

In [17]:
avg_review_score_per_state_agg = merged_df.groupby('customer_state')['review_score'].agg('mean').reset_index()
avg_review_score_per_state_agg.rename(columns={'review_score': 'average_review_score'}, inplace=True)

🤩 `.agg()` is much more flexible than the other methods, push it further!

In [20]:
agg_functions = {
    'mean': 'mean',  # Calculate the average
    'count': 'count',  # Count the number of entries
    'max': 'max',  # Find the maximum review score
    'min': 'min',  # Find the minimum review score
    'std': 'std'  # Compute the standard deviation
}

review_score_stats_per_state = merged_df.groupby('customer_state')['review_score'].agg(['mean', 'count', 'max', 'min', 'std']).reset_index()
review_score_stats_per_state.columns = ['customer_state', 'average_review_score', 'review_count', 'max_review_score', 'min_review_score', 'std_dev_review_score']
print(review_score_stats_per_state)

   customer_state  average_review_score  review_count  max_review_score  \
0              AC              4.049383            81               5.0   
1              AL              3.751208           414               5.0   
2              AM              4.183673           147               5.0   
3              AP              4.194030            67               5.0   
4              BA              3.860888          3357               5.0   
5              CE              3.851016          1329               5.0   
6              DF              4.064711          2148               5.0   
7              ES              4.041667          2016               5.0   
8              GO              4.042490          2024               5.0   
9              MA              3.764075           746               5.0   
10             MG              4.136172         11625               5.0   
11             MS              4.118785           724               5.0   
12             MT        

### NPS per State

❓Now, it is time to create a 🔥 **custom aggregation function** to compute the `NPS per customer_state` directly.

1️⃣ Create your `nps` function

2️⃣ Try to debug it using the `breakpoint()` debugger within your function to understand clearly what objects you are manipulating

<br>

💡 *PS.:* always **cleanly** exit your debugger by typing `exit` when inside the debugging session, otherwise you will have to restart your Notebook!

In [21]:
import pandas as pd


def calculate_nps(scores):
    promoters = scores[scores == 5].count()
    passives = scores[scores == 4].count()
    detractors = scores[scores <= 2].count()

    nps = ((promoters - detractors) / (promoters + passives + detractors)) * 100
    return nps
nps_per_state = merged_df.groupby('customer_state')['review_score'].apply(calculate_nps).reset_index(name='NPS')
print(nps_per_state)

   customer_state        NPS
0              AC  43.835616
1              AL  25.454545
2              AM  47.517730
3              AP  49.152542
4              BA  33.377483
5              CE  33.973289
6              DF  44.843276
7              ES  44.111232
8              GO  44.189853
9              MA  28.422619
10             MG  49.784159
11             MS  48.507463
12             MT  47.030303
13             PA  33.830275
14             PB  43.685300
15             PE  42.574257
16             PI  36.748330
17             PR  51.320593
18             RJ  36.568502
19             RN  47.855530
20             RO  45.777778
21             RR  23.076923
22             RS  49.106079
23             SC  46.214416
24             SE  31.562500
25             SP  51.756922
26             TO  44.531250


👉 Now, use your `nps` function to compute the `NPS per customer_state`.

In [22]:
def calculate_nps(scores):
    promoters = scores[scores == 5].count()
    passives = scores[scores == 4].count()
    detractors = scores[scores <= 2].count()
    
    nps = ((promoters - detractors) / (promoters + passives + detractors)) * 100
    return nps
nps_per_state = merged_df.groupby('customer_state')['review_score'].apply(calculate_nps).reset_index(name='NPS')
print(nps_per_state)

   customer_state        NPS
0              AC  43.835616
1              AL  25.454545
2              AM  47.517730
3              AP  49.152542
4              BA  33.377483
5              CE  33.973289
6              DF  44.843276
7              ES  44.111232
8              GO  44.189853
9              MA  28.422619
10             MG  49.784159
11             MS  48.507463
12             MT  47.030303
13             PA  33.830275
14             PB  43.685300
15             PE  42.574257
16             PI  36.748330
17             PR  51.320593
18             RJ  36.568502
19             RN  47.855530
20             RO  45.777778
21             RR  23.076923
22             RS  49.106079
23             SC  46.214416
24             SE  31.562500
25             SP  51.756922
26             TO  44.531250


Again, instead of using this function, try to do the same task in one line of code, remember the `KISS` principle? 😉

In [23]:
nps_per_state = (merged_df.groupby('customer_state')['review_score']
                 .apply(lambda x: ((x == 5).sum() - (x <= 2).sum()) / x.count() * 100)
                 .reset_index(name='NPS'))

# Cheat Sheet for `map`, `apply`, `applymap` and `groupby`

```python
# MAP (for Series)
series.map(function) 
series.map({mapping dict})

# APPLY (for DataFrame)
df.apply(lambda col: col.max(), axis = 0)     # default axis
df.apply(lambda row: row[‘A’] + row[‘B’], axis = 1)

df.applymap(my_funct_for_indiv_elements)
df.applymap(lambda x: '%.2f' % x)
```

```python
## GROUPBY
group = df.groupby('col_A')

group.mean()
group.apply(np.mean)
group.agg({
    col_A: ['mean', np.sum],
    col_B: my_custom_sum,
    col_B: lambda s: my_custom_sum(s)
})

group.apply(custom_mean_function)
```

[Introduction to Pandas' `apply`, `applymap` and `map` - Towards Data Science](https://towardsdatascience.com/introduction-to-pandas-apply-applymap-and-map-5d3e044e93ff)